## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
import os
city_data_df = pd.read_csv(os.path.join('Weather_Database', 'WeatherPy_database.csv'))
city_data_df.head()

#import numpy as np
#print (f"Min: {np.min(city_data_df['Max Temp'])}, Max: {np.max(city_data_df['Max Temp'])}")

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,55.09,82,90,8.19,overcast clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,78.76,88,56,5.99,broken clouds
2,2,Upernavik,GL,72.7868,-56.1549,31.21,94,94,16.31,overcast clouds
3,3,Atuona,PF,-9.8000,-139.0333,78.66,71,14,11.01,few clouds
4,4,Tiarei,PF,-17.5333,-149.3333,75.22,77,43,12.33,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
import numbers

min_temp = 0
max_temp = 0

while True:
    min_temp = input('Enter minimum temperature:')
    max_temp = input('Enter maximum temperature:')
    
    # Convert to floats:
    try:
        min_temp = float(min_temp)
        max_temp = float(max_temp)
    except:
        print ('Please enter valid numbers for min temp and max temp, try again!')
        continue
        
    if min_temp > max_temp:
        print ('Min temp must be less than or equal to the max temp, try again!')
        continue
        
    print (f"You entered min temp as {min_temp} and max temp as {max_temp}")
    break

Enter minimum temperature:30
Enter maximum temperature:80
You entered min temp as 30.0 and max temp as 80.0


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & 
                            (city_data_df['Max Temp'] <= max_temp)]
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,55.09,82,90,8.19,overcast clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,78.76,88,56,5.99,broken clouds
2,2,Upernavik,GL,72.7868,-56.1549,31.21,94,94,16.31,overcast clouds
3,3,Atuona,PF,-9.8000,-139.0333,78.66,71,14,11.01,few clouds
4,4,Tiarei,PF,-17.5333,-149.3333,75.22,77,43,12.33,light rain
...,...,...,...,...,...,...,...,...,...,...
685,685,Tianpeng,CN,30.9866,103.9393,73.63,73,100,5.97,overcast clouds
686,686,Kigonsera,TZ,-10.8000,35.0500,63.37,64,7,2.26,clear sky
689,689,Zadar,HR,44.1197,15.2422,74.01,35,0,12.66,clear sky
694,694,Ozernovskiy,RU,51.5000,156.5167,40.77,84,29,2.64,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.

# NOTE. The usual methods to locate NAN values (such as as shown in section 6.5.3) do not work on 
# this data set as the country code NA (for Namibia) is incorrectly recognized as a NULL value. 
# This is wrong. Another approach is used below to get all IDs of cities with specific criteria 
# applied to check null for each column.

ids = clean_df.loc[
                    (clean_df['City'] == '') |
                    (clean_df['Country'] == '') |
                    (pd.isna(clean_df['Lat']) == True) |
                    (pd.isna(clean_df['Lng']) == True) |
                    (pd.isna(clean_df['Max Temp']) == True) |
                    (pd.isna(clean_df['Humidity']) == True) |
                    (pd.isna(clean_df['Cloudiness']) == True) |
                    (pd.isna(clean_df['Wind Speed']) == True) |
                    (clean_df['Current Description'] == '')
                ]['City_ID'].tolist()

print (ids)

[]


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.drop(labels=ids, axis=0)
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,55.09,82,90,8.19,overcast clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,78.76,88,56,5.99,broken clouds
2,2,Upernavik,GL,72.7868,-56.1549,31.21,94,94,16.31,overcast clouds
3,3,Atuona,PF,-9.8000,-139.0333,78.66,71,14,11.01,few clouds
4,4,Tiarei,PF,-17.5333,-149.3333,75.22,77,43,12.33,light rain
...,...,...,...,...,...,...,...,...,...,...
685,685,Tianpeng,CN,30.9866,103.9393,73.63,73,100,5.97,overcast clouds
686,686,Kigonsera,TZ,-10.8000,35.0500,63.37,64,7,2.26,clear sky
689,689,Zadar,HR,44.1197,15.2422,74.01,35,0,12.66,clear sky
694,694,Ozernovskiy,RU,51.5000,156.5167,40.77,84,29,2.64,scattered clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,55.09,overcast clouds,-33.6500,115.3333,
1,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,
2,Upernavik,GL,31.21,overcast clouds,72.7868,-56.1549,
3,Atuona,PF,78.66,few clouds,-9.8000,-139.0333,
4,Tiarei,PF,75.22,light rain,-17.5333,-149.3333,
5,Hasaki,JP,68.29,scattered clouds,35.7333,140.8333,
7,Cherskiy,RU,47.10,few clouds,68.7500,161.3000,
8,Khatanga,RU,38.03,overcast clouds,71.9667,102.5000,
10,New Norfolk,AU,45.90,scattered clouds,-42.7826,147.0587,
11,Provideniya,RU,38.07,clear sky,64.3833,-173.3000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params['location'] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    try:
        hotels = requests.get(base_url, params=params).json()
            
        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        try:
            hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
            #print (f"Index: {index}, City: {row['City']}, Hotel name: {hotels['results'][0]['name']}")
        
        except (IndexError):
            print (f"Index: {index}, City: {row['City']}, No hotel! Skipping.")
    
    except Exception as e:
        print (f"Index: {index}, City: {row['City']}, response message {hotels['message']}")
        
print ('\n-----------------------------')
print ('Done getting all hotels!')
print ('-----------------------------')

Index: 11, City: Provideniya, No hotel! Skipping.
Index: 15, City: Vostok, No hotel! Skipping.
Index: 66, City: Albany, No hotel! Skipping.
Index: 84, City: Koygorodok, No hotel! Skipping.
Index: 94, City: Srednekolymsk, No hotel! Skipping.
Index: 107, City: Sangar, No hotel! Skipping.
Index: 118, City: Ngerengere, No hotel! Skipping.
Index: 144, City: Saskylakh, No hotel! Skipping.
Index: 148, City: Aksarka, No hotel! Skipping.
Index: 158, City: Chokurdakh, No hotel! Skipping.
Index: 164, City: Kungurtug, No hotel! Skipping.
Index: 167, City: Teguldet, No hotel! Skipping.
Index: 192, City: Belaya Gora, No hotel! Skipping.
Index: 207, City: Ust-Shonosha, No hotel! Skipping.
Index: 228, City: Ostrovnoy, No hotel! Skipping.
Index: 269, City: Naziya, No hotel! Skipping.
Index: 284, City: Ust-Maya, No hotel! Skipping.
Index: 299, City: Chunoyar, No hotel! Skipping.
Index: 311, City: Sobolevo, No hotel! Skipping.
Index: 315, City: Cheremnoye, No hotel! Skipping.
Index: 318, City: Uvinza, No

In [17]:
# 7. Drop the rows where there is no Hotel Name.
ids = hotel_df.loc[(hotel_df['Hotel Name'] == '')].index.tolist()
clean_hotel_df = hotel_df.drop(labels=ids, axis=0)
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,55.09,overcast clouds,-33.6500,115.3333,Observatory Guest House
1,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
2,Upernavik,GL,31.21,overcast clouds,72.7868,-56.1549,Gina's Guesthouse - Upernavik
3,Atuona,PF,78.66,few clouds,-9.8000,-139.0333,Villa Enata
4,Tiarei,PF,75.22,light rain,-17.5333,-149.3333,Le Rocher de Tahiti
...,...,...,...,...,...,...,...
682,Sept-Iles,CA,54.12,broken clouds,50.2001,-66.3821,Comfort Inn
683,Farafangana,MG,73.31,few clouds,-22.8167,47.8333,Miarina
685,Tianpeng,CN,73.63,overcast clouds,30.9866,103.9393,Shuangliu Love Home Business Hotel
689,Zadar,HR,74.01,clear sky,44.1197,15.2422,Hotel Kolovare


In [18]:
import os

# 8a. Create the output File (CSV)
output_data_file = os.path.join('Vacation_Search', 'WeatherPy_vacation.csv')

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
max_temp = clean_hotel_df['Max Temp']
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))